In [1]:
import langchain
import os
import dotenv
dotenv.load_dotenv()

True

# Load documents

In [2]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("../data/text.pdf")
docs = loader.load()
print(len(docs))
type(docs[0])
print(docs[0].metadata)
print(docs[20].page_content[:500])

52
{'source': '../data/text.pdf', 'page': 0}
Capítulo 2. Fundamentação Teórica 20
Figura 2 – Mecanismo de atenção
Fonte: Adaptado de Kamath, Graham e Emara (2022).
A arquitetura Transformer utiliza 6 codificadores e 6 decodificadores em para-
lelo (Figura 3), cada um com os seus mecanismos próprios de Auto-Atenção (VASWANI
et al. , 2017). Além disso, Kamath, Graham e Emara (2022) apontam que a arquitetura
utiliza diversos "truques" para aumentar a sua capacidade de aprendizado, entre eles:
o mecanismo de Auto-Atenção é repetido 8 vezes par


In [3]:
docs += PyPDFLoader("../data/slides.pdf").load()
docs += PyPDFLoader("../data/annotations.pdf").load()

# Split

In [4]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=500,
    chunk_overlap=100,
    length_function=len
)

In [5]:
docs_splitted = text_splitter.split_documents(docs)
print(len(docs))
print(len(docs_splitted))
print('-------')
print(type(docs_splitted[50]))
print(docs_splitted[50].metadata)
print(docs_splitted[50].page_content)

80
288
-------
<class 'langchain.schema.document.Document'>
{'source': '../data/text.pdf', 'page': 12}
viabilizar a implementação de um software que atenda, de forma satisfatória, o analista
de suporte, por consequência, melhorando o serviço prestado aos usuários de TI.
1.2 Definição do Problema
Muitos chamados possuem resolução idêntica, de forma que basta ao ana-
lista identificar se um problema similar já foi resolvido no passado (MUNI et al. , 2017).
Para isso, o analista pode perguntar a colegas, ler históricos de conversas, buscar dire-


# Vectorization and indexing

In [15]:
os.environ["OPENAI_API_BASE"] = os.getenv('OPENAI_AZURE_ENDPOINT')
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_AZURE_KEY')
os.environ["OPENAI_API_VERSION"] = "2023-05-15"
os.environ["OPENAI_API_TYPE"] = "azure"

from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(
    deployment = 'text-embedding-ada-002',
    chunk_size = 1,
)

In [11]:
# Example
sentence1 = "i like dogs"
embedding1 = embedding.embed_query(sentence1)

print(type(embedding1))
print(len(embedding1))
print(type(embedding1[0]))
print(embedding1[:10])

<class 'list'>
1536
<class 'float'>
[-0.027588920667767525, -0.005362407304346561, -0.025769870728254318, -0.03319765627384186, -0.0272857453674078, 0.022536005824804306, -0.010326896794140339, -0.008280466310679913, 0.0025564597453922033, -0.01980742998421192]


# Persist

In [7]:
from langchain.vectorstores import Chroma
persist_directory = '../assets/persistence/chroma/'

In [8]:
#os.system(f"rm -rf {persist_directory}") # Careful: this deletes the DB

0

In [16]:
# This will already calculate the embeddings for all documents
# That is, many calls to OpenAI, be aware of costs
vectordb = Chroma.from_documents(
    documents=docs_splitted,
    embedding=embedding,
    persist_directory=persist_directory
)

In [17]:
vectordb.persist()